In [139]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**Loaded data and droping unnessary column **

In [140]:
data=pd.read_csv('/content/thyroid_cancer_risk_data.csv')
data.drop(['Patient_ID','Country'] ,axis=1,inplace=True)
df=pd.DataFrame(data)
df.head()

,Age,Gender,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,66,Male,Caucasian,No,Yes,No,No,No,No,9.37,1.67,6.16,1.08,Low,Benign
1,29,Male,Hispanic,No,Yes,No,No,No,No,1.83,1.73,10.54,4.05,Low,Benign
2,86,Male,Caucasian,No,No,No,No,No,No,6.26,2.59,10.57,4.61,Low,Benign
3,75,Female,Asian,No,No,No,No,No,No,4.10,2.62,11.04,2.46,Medium,Benign
4,35,Female,African,Yes,Yes,No,No,No,No,9.10,2.11,10.71,2.11,High,Benign


Filling missing values in data columns using sklearn

In [141]:
for col in df.columns:
 # df[col]=df[col].fillna(df[col].mode())
  from sklearn.impute import SimpleImputer
  s=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
  df[col]=s.fit_transform(df[[col]]).flatten()

Converting object data type into category dtype  for better efficiency and operation.

In [142]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

using ordinal encoding because of it has more than 2 category

In [143]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df["Ethnicity"] = encoder.fit_transform(df[["Ethnicity"]])

Here, i used labelencoder bcauz it has 2 option value category

In [144]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in df.select_dtypes(include=['category']).columns:
    df[col] = le.fit_transform(df[col])

Data Analysis by  using tool ydata profiling ref: [](https://)https://docs.profiling.ydata.ai/latest/

In [ ]:
from ydata_profiling import ProfileReport
prof=ProfileReport(df)
prof.to_file(output_file ='output.html')

Train test split

In [154]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["Thyroid_Cancer_Risk", "Diagnosis"])  # Features
y = df[["Thyroid_Cancer_Risk", "Diagnosis"]]
X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=5)

Scalling the data for equal magnitutude

In [156]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
x_test=sc.fit_transform(x_test)

using multioutputclass bcuauz the model has two targrt value
and the XCB algo

In [178]:
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
model=MultiOutputClassifier(XGBClassifier())
model.fit(X_train,y_train)
y_pred=model.predict(x_test)

Evalution and testing here we can check f1_score as well but the model has 2 target value so i used Classification report

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = model.predict(x_test)

# Convert predictions into DataFrame (same columns as y_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# Accuracy for each target variable
accuracy_risk = accuracy_score(y_test["Thyroid_Cancer_Risk"], y_pred_df["Thyroid_Cancer_Risk"])
accuracy_diagnosis = accuracy_score(y_test["Diagnosis"], y_pred_df["Diagnosis"])

# Classification reports
report_risk = classification_report(y_test["Thyroid_Cancer_Risk"], y_pred_df["Thyroid_Cancer_Risk"])
report_diagnosis = classification_report(y_test["Diagnosis"], y_pred_df["Diagnosis"])

# Print results
print("Accuracy (Thyroid Cancer Risk):", accuracy_risk)
print("Classification Report (Thyroid Cancer Risk):\n", report_risk)

print("\nAccuracy (Diagnosis):", accuracy_diagnosis)
print("Classification Report (Diagnosis):\n", report_diagnosis)

conclusion:
apperanty the model has avg confidence score but later i will try to increase it's Accuracy (Thyroid Cancer Risk): 0.5755704613841525
Classification Report (Thyroid Cancer Risk):
               precision    recall  f1-score   support

           0       0.98      0.51      0.67      9580
           1       0.55      0.96      0.70     32356
           2       0.36      0.04      0.07     21872

    accuracy                           0.58     63808
   macro avg       0.63      0.50      0.48     63808
weighted avg       0.55      0.58      0.48     63808


Accuracy (Diagnosis): 0.7974705366098295
Classification Report (Diagnosis):
               precision    recall  f1-score   support

           0       0.81      0.97      0.88     49020
           1       0.69      0.23      0.34     14788

    accuracy                           0.80     63808
   macro avg       0.75      0.60      0.61     63808
weighted avg       0.78      0.80      0.76     63808